In [1]:
from pycaret.classification import *

import pandas as pd
import numpy as np
import re
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

### Load the data

In [2]:
#df = pd.read_csv("./intermediate_data/.csv", encoding = 'latin')
#df = pd.read_excel("./intermediate_data/.xlsx, engine='openpyxl')
df = pd.read_csv("./intermediate_data/proof_of_concept.csv", encoding = 'latin')


In [3]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id_inv  23 non-null     object 
 1   y_true  23 non-null     object 
 2   Mg      23 non-null     float64
 3   Al      23 non-null     float64
 4   Si      23 non-null     float64
 5   P       23 non-null     float64
 6   S       23 non-null     float64
 7   Cl      23 non-null     float64
 8   K       23 non-null     float64
 9   Ca      23 non-null     float64
 10  Sc      23 non-null     float64
 11  Ti      23 non-null     float64
 12  V       23 non-null     float64
 13  Cr      23 non-null     float64
 14  Mn      23 non-null     float64
 15  Fe      23 non-null     float64
 16  Co      23 non-null     float64
 17  Ni      23 non-null     float64
 18  Cu      23 non-null     float64
 19  Zn      23 non-null     float64
 20  Ga      23 non-null     float64
 21  Ge      23 non-null     float64
 22  As  

#### This function load the pre-trained models, perform a prediction on the data and format the output dataframe

In [4]:
MODEL_NAMES = {
    'model1': 'strunz_classifier',
    'model2': 'classifier'
}

def prediction_function(df, model_key):
    """
    This function performs the following operations:
    * Loads the specified pre-trained model
    * Performs the probability prediction on the dataframe (df).
    * Filters the columns containing the word 'Score' for output formatting purposes
    * Selects the three highest probabilities, sorts them in descending order and presents them in a new column.
    * Finally it returns the dataframe with two new columns of results 'Label' and 'top_3 Probs'.
    """
    
    model = load_model(MODEL_NAMES[model_key])
    pred_prob = predict_model(estimator=model, data=df, raw_score=True)
    
    selected = [] 
    patron = re.compile('.*Score.*', re.IGNORECASE)
    
    for columna in pred_prob.columns:
        if patron.match(columna):
            selected.append(columna)
    
    def select_top_three_cols(row):
        top_three_cols = row.nlargest(3)
        top_three_cols_dict = dict(zip(top_three_cols.index, top_three_cols))
        return top_three_cols_dict
    
    top_three_cols = pred_prob[selected].apply(select_top_three_cols, axis=1)
    pred_prob['top_3_probs'] = top_three_cols
    
    pred_prob = pred_prob.drop(pred_prob[selected].columns, axis=1)
     
    return pred_prob

### Using the prediction function with the two models

In [5]:
# Model 1
preds = prediction_function(df, 'model1')
preds.rename(columns={'prediction_label': 'Predicted Major Class', 'top_3_probs': 'top_3_Major Class'}, inplace=True)
preds = preds.drop(['top_3_Major Class'], axis=1)


Transformation Pipeline and Model Successfully Loaded


In [6]:
# Model 2
preds2 = prediction_function(preds, 'model2')
preds2

Transformation Pipeline and Model Successfully Loaded


,id_inv,y_true,Mg,Al,Si,P,S,Cl,K,Ca,...,Hg,Tl,Pb,Th,xrd1,xrd2,xrd3,Predicted Major Class,prediction_label,top_3_probs
0,MRO_1581,Clinochlore,10.536120,9.313109,44.244869,1.597565,1.997287,6.036622,0.000165,7.068465,...,0.000003,0.000003,0.000003,0.000003,Clinochlore,NaN,NaN,Silicates,Clinochlore,"{'prediction_score_Clinochlore': 0.9637, 'pred..."
1,MRO_1836,Muscovite,0.000000,28.897903,40.323185,1.208307,0.466497,2.163752,21.502310,1.166194,...,0.000003,0.000003,0.000003,0.000003,Muscovite,NaN,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.5687, 'predic..."
2,MRO_1890,Muscovite,0.000000,29.416237,35.391479,1.151213,0.874422,2.691249,23.828667,0.899188,...,0.000003,0.000003,0.000003,0.000003,Muscovite,NaN,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.9622, 'predic..."
3,MRO_1985A,Clinochlore,0.000000,19.010382,22.319599,8.575088,2.722408,14.522683,0.000157,11.104357,...,0.000003,0.000003,0.000003,0.000003,Clinochlore,NaN,NaN,Silicates,Clinochlore,"{'prediction_score_Clinochlore': 0.5361, 'pred..."
4,MRO_1999,Clinochlore,0.000000,23.779680,24.164476,6.535106,2.098606,10.985693,0.914940,5.413211,...,0.000003,0.000003,0.000003,0.000003,Clinochlore,NaN,NaN,Silicates,Chlorite-serpentine,{'prediction_score_Chlorite-serpentine': 0.961...
5,MRO(N)2,Muscovite,0.000000,29.348356,34.880253,1.071741,0.593321,2.478651,18.738752,9.104530,...,0.000003,0.000003,0.000003,0.000003,Muscovite,NaN,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.9644, 'predic..."
6,MRO(N)3,Muscovite,0.000000,28.147934,36.798157,1.293025,0.649161,2.589902,23.795195,1.525216,...,0.000003,0.000003,0.000003,0.000003,Muscovite,NaN,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.9643, 'predic..."
7,MRO(N)5,Muscovite,0.000000,34.303707,36.722488,1.094201,0.654560,2.764978,20.582144,0.642881,...,0.000003,0.000003,0.000003,0.000003,Muscovite,NaN,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.966, 'predict..."
8,CLP_131,Talc,19.685595,1.013071,63.269554,0.672530,0.397608,2.962955,0.000162,0.934872,...,0.000003,0.000003,0.000003,0.000003,Talc,NaN,NaN,Silicates,Talc,"{'prediction_score_Talc': 0.9541, 'prediction_..."
9,CLP_133,Clinochlore,0.000000,25.889153,32.451939,1.216468,0.556327,2.462242,9.730409,1.077782,...,0.000003,0.000003,0.000003,0.000003,Clinochlore,Phlogopite,NaN,Silicates,Muscovite,"{'prediction_score_Muscovite': 0.9511, 'predic..."


#### The dataframe with predictions is stored in intermediate_data folder

In [7]:

preds2.to_csv("./intermediate_data/proof_of_concept_predictions.csv") 

### Evaluation

**In this section we report the confusion matrix and the classification report for the proof of concept.**

In [8]:
preds2['y_pred'] = preds2['prediction_label']

predictions_filtered = preds2[['id_inv', 'y_true', 'y_pred']]

predictions_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   id_inv  23 non-null     category
 1   y_true  23 non-null     category
 2   y_pred  23 non-null     object  
dtypes: category(2), object(1)
memory usage: 1.2+ KB


In [9]:
y_true = predictions_filtered['y_true']
y_pred = predictions_filtered['y_pred']
etiquetas_true = y_true.unique().tolist() #
etiquetas_pred = y_pred.unique().tolist()

y_pred.nunique()

5

In [10]:
etiquetas = etiquetas_pred + etiquetas_true
etiquetas

['Clinochlore',
 'Muscovite',
 'Chlorite-serpentine',
 'Talc',
 'Variscite',
 'Clinochlore',
 'Muscovite',
 'Talc']

In [11]:
# Combinar las dos series en una sola serie
etiquetas_combinadas = pd.concat([y_true, y_pred], ignore_index=True)

# Obtener los elementos únicos de la serie combinada
etiquetas_unicas = np.unique(etiquetas_combinadas)

# Crear una nueva serie con los elementos únicos
unique_series = pd.Series(etiquetas_unicas)

# Mostrar los resultados
unique_series.nunique()

5

In [12]:
cm = confusion_matrix(y_true, y_pred, labels=unique_series)
cm_display = ConfusionMatrixDisplay(cm, display_labels=unique_series)
fig, ax = plt.subplots(figsize=(10, 10))
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.title('Confusion Matrix', fontsize=18)
plt.xlabel('Etiqueta predicha', fontsize=15)
plt.ylabel('Etiqueta verdadera', fontsize=15)
cm_display.plot(ax=ax,cmap='binary')
cm_display.ax_.set_xticklabels(unique_series, rotation=45, ha='right', fontdict={'fontsize': 10})
cm_display.ax_.set_yticklabels(unique_series, fontdict={'fontsize': 10})

plt.show()

NameError: name 'plt' is not defined